In [1]:
data_path = "./data/"
sp_data_path = data_path + "SP-train.npy"
wp_data_path = data_path + "WP-train.npy"

In [11]:
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

# do this once
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

/home/qianqi/.conda/envs/nlp243/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sp_train = np.load(sp_data_path)
wp_train = np.load(wp_data_path)

In [34]:
print(len(sp_train))
for k, v in sp_train[0].items():
    print(k, v)
print()
print(len(wp_train))
for k, v in wp_train[0].items():
    print(k, v)

'''
data has two ids 
SP = sentence puzzle, WP = word puzzle 

each data point has a question, an answer, and a list of the choices, 
a lable indicating the index of the correct choice, and a choice order (which we probably won't use)
'''


507
id SP-0
question Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?
answer Each daughter shares the same brother.
distractor1 Some daughters get married and have their own family.
distractor2 Some brothers were not loved by family and moved away.
distractor(unsure) None of above.
label 1
choice_list ['Some daughters get married and have their own family.', 'Each daughter shares the same brother.', 'Some brothers were not loved by family and moved away.', 'None of above.']
choice_order [1, 0, 2, 3]

396
id WP-0
question How do you spell COW in thirteen letters?
answer SEE O DOUBLE YOU.
distractor1 COWCOWCOWCOWW
distractor2 SEE OH DEREFORD
distractor(unsure) None of above.
label 1
choice_list ['SEE OH DEREFORD', 'SEE O DOUBLE YOU.', 'COWCOWCOWCOWW', 'None of above.']
choice_order [2, 0, 1, 3]


In [12]:
'''
Trying out a pretrained BERT model for QA 
'''
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

config.json: 100%|██████████| 443/443 [00:00<00:00, 75.6kB/s]
model.safetensors: 100%|██████████| 1.34G/1.34G [00:20<00:00, 65.1MB/s]
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 6.31kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.46MB/s]
tokenizer.json: 100%|██████████| 466k/46

In [64]:
'''
Start scores is an output of the same length as the context paragraph given, 
and each label indicates the probability that the answer starts at that word embedding
'''

question = sp_train[10]['question']
paragraph = f"A. {sp_train[10]['answer']}\nB. {sp_train[10]['distractor1']}\nC. {sp_train[10]['distractor2']}\nD. {sp_train[10]['distractor(unsure)']}"
            
encoding = tokenizer.encode_plus(text=question,text_pair=paragraph)
inputs = encoding['input_ids']
sentence_embedding = encoding['token_type_ids'] 
tokens = tokenizer.convert_ids_to_tokens(inputs) 
outputs = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
start_scores = outputs.start_logits
end_scores = outputs.end_logits
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)
answer = ' '.join(tokens[start_index : end_index+1])

In [26]:
def func(*list_input):
    return sum(list_input)
def func2(**dict_input):
    # print(dict_input)
    num_image = dict_input['num_image']
    return num_image

In [28]:
# print(func([1,2,3,4,5]))
# print(func2({'wili':1, 'qq':2}))
# print(func(1,2,3,4))
model_name = "adrgagjaopgkjpaokgpoar"
print(func2(num_image=10, model=model_name))


10


In [65]:
print(question)
print(paragraph)
print(answer)

There are five apples in a basket. You distribute them among the five girls, giving each one an apple while leaving one apple in the basket. How is that even doable?
A. Give the fifth girl her apple in the basket.
B. Two girls decide to share one apple.
C. One girl is polite and return her apple back.
D. None of above.
give the fifth girl her apple in the basket .


In [56]:
question = wp_train[0]['question']
paragraph = f"A. {wp_train[0]['answer']}\nB. {wp_train[0]['distractor1']}\nC. {wp_train[0]['distractor2']}\nD. {wp_train[0]['distractor(unsure)']}"
            
encoding = tokenizer.encode_plus(text=question,text_pair=paragraph)
inputs = encoding['input_ids']
sentence_embedding = encoding['token_type_ids'] 
tokens = tokenizer.convert_ids_to_tokens(inputs) 
outputs = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
start_scores = outputs.start_logits
end_scores = outputs.end_logits
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)
answer = ' '.join(tokens[start_index : end_index+1])

In [55]:
print(tokens)
print(question)
print(paragraph)

['[CLS]', 'how', 'do', 'you', 'spell', 'cow', 'in', 'thirteen', 'letters', '?', '[SEP]', 'a', '.', 'see', 'o', 'double', 'you', '.', 'b', '.', 'cow', '##co', '##wc', '##ow', '##co', '##w', '##w', 'c', '.', 'see', 'oh', 'der', '##ef', '##ord', 'd', '.', 'none', 'of', 'above', '.', '[SEP]']
How do you spell COW in thirteen letters?
A. SEE O DOUBLE YOU.
B. COWCOWCOWCOWW
C. SEE OH DEREFORD
D. None of above.


In [54]:
print(answer)

cow ##co ##wc ##ow ##co ##w ##w


In [57]:
question = wp_train[1]['question']
paragraph = f"A. {wp_train[1]['answer']}\nB. {wp_train[1]['distractor1']}\nC. {wp_train[1]['distractor2']}\nD. {wp_train[1]['distractor(unsure)']}"
            
encoding = tokenizer.encode_plus(text=question,text_pair=paragraph)
inputs = encoding['input_ids']
sentence_embedding = encoding['token_type_ids'] 
tokens = tokenizer.convert_ids_to_tokens(inputs) 
outputs = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
start_scores = outputs.start_logits
end_scores = outputs.end_logits
start_index = torch.argmax(start_scores)

end_index = torch.argmax(end_scores)
answer = ' '.join(tokens[start_index : end_index+1])

In [58]:
print(tokens)
print(question)
print(paragraph)

['[CLS]', 'in', 'thirteen', 'letters', ',', 'how', 'do', 'you', 'spell', 'cow', '?', '[SEP]', 'a', '.', 'see', 'o', 'double', 'you', '.', 'b', '.', 'cow', '##co', '##wc', '##ow', '##co', '##w', '##w', 'c', '.', 'see', 'oh', 'der', '##ef', '##ord', 'd', '.', 'none', 'of', 'above', '.', '[SEP]']
In thirteen letters, how do you spell COW?
A. SEE O DOUBLE YOU.
B. COWCOWCOWCOWW
C. SEE OH DEREFORD
D. None of above.


In [59]:
print(answer)

cow ##co ##wc ##ow ##co ##w ##w


In [ ]:
print(wp_train[0])


In [60]:
question = wp_train[5]['question']
paragraph = f"A. {wp_train[5]['answer']}\nB. {wp_train[5]['distractor1']}\nC. {wp_train[5]['distractor2']}\nD. {wp_train[5]['distractor(unsure)']}"
            
encoding = tokenizer.encode_plus(text=question,text_pair=paragraph)
inputs = encoding['input_ids']
sentence_embedding = encoding['token_type_ids'] 
tokens = tokenizer.convert_ids_to_tokens(inputs) 
outputs = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
start_scores = outputs.start_logits
end_scores = outputs.end_logits
start_index = torch.argmax(start_scores)

end_index = torch.argmax(end_scores)
answer = ' '.join(tokens[start_index : end_index+1])

In [61]:
print(tokens)
print(question)
print(paragraph)
print(answer)

['[CLS]', 'what', 'does', 'eight', 'plus', 'three', 'equal', 'if', 'ten', 'plus', 'three', 'equals', 'one', '?', '[SEP]', 'a', '.', 'eleven', '.', 'b', '.', 'two', '.', 'c', '.', 'five', '.', 'd', '.', 'none', 'of', 'above', '.', '[SEP]']
What does eight plus three equal if ten plus three equals one?
A. Eleven.
B. Two.
C. Five.
D. None of above.
two . c . five
